In [87]:
# Big Data Final Project - Analisis de Indicadores Ninez + Acceso al Internet

In [88]:
# import modules
import os
import pandas as pd

In [89]:
# assign path
path, dirs, files1 = next(os.walk("./data/indicadores_nines/"))
file_count = len(files1)

In [90]:
# create empty list
dataframes_list_indicadores_nines = []

In [91]:
# append datasets to the list
for i in range(file_count):
	temp_df = pd.read_csv("./data/indicadores_nines/"+files1[i], delimiter=';')
	dataframes_list_indicadores_nines.append(temp_df)

In [92]:
# display datasets
count = 0
for dataset in dataframes_list_indicadores_nines:
    display(dataset.style.set_table_attributes("style=display:inline").set_caption(files1[count]))
    count+=1

,Grupos de edad,Total Pais,Tiene,No tiene
0,Total,2.511.100,"92,3","7,7"
1,0 a 4 anos,667.241,"77,6","22,4"
2,5 a 9 anos,720.501,97,3
3,10 a 14 anos,685.305,"97,9","2,1"
4,15 a 17 anos,438.053,"98,5","-1,5"


,Sexo y grupos de edad,Total pais,Utilizo Internet,No utilizo Internet
0,Poblacion Total,1.123.358,"71,2","28,8"
1,10 a 14 anos (Total),685.305,"61,3","38,7"
2,15 a 17 anos (Total),438.053,"86,6","13,4"
3,Poblacion Hombres,598.477,"69,9","30,1"
4,10 a 14 anos (Hombres),369.419,61,39
5,15 a 17 anos (Hombres),229.058,"84,4","15,6"
6,Poblacion Mujeres,524.881,"72,6","27,4"
7,10 a 14 anos (Mujeres),315.886,"61,7","38,3"
8,15 a 17 anos (Mujeres),208.995,89,11


,Seguro medico,Total Pais,Urbana,Rural
0,Total,2.511.100,1.477.574,1.033.526
1,Tiene,"22,9","30,1","12,6"
2,No tiene,"77,1","69,9","87,4"


,Grupos de edad,Total Pais,Asiste,No asiste
0,Total,1.843.859,"92,8","7,2"
1,5 a 9 anos,720.501,95,5
2,10 a 14 anos,685.305,"96,4","3,6"
3,15 a 17 anos,438.053,"83,5","16,5"


,Indicador de la fuerza laboral,Total,Hombres,Mujeres
0,Poblacion de 15 a 17 anos,438.053000,229.058000,208.995000
1,Poblacion en la fuerza de trabajo,146.875000,100.952000,45.923000
2,Poblacion Ocupada,130.906000,90.865000,40.041000
3,Poblacion desocupada,15.969000,10.087000,5.882000
4,Poblacion fuera de la fuerza de trabajo,291.178000,128.106000,163.072000


...

,Grupos de edad,Total Pais,Enfermo / accidentado,Sano
0,Total,2.511.100,"15,9","84,1"
1,0 a 4 anos,667.241,21,79
2,5 a 9 anos,720.501,"17,9","82,1"
3,10 a 14 anos,685.305,"10,8","89,2"
4,15 a 17 anos,438.053,13,87


,Grupos de edad,Total Pais,Pobre extremo,Pobre no extremo,No pobre
0,Total,2.508.328,"5,9","30,5","63,7"
1,0 a 4 anos,666.801,"5,5","31,3","63,3"
2,5 a 9 anos,720.246,"5,8","30,4","63,8"
3,10 a 14 anos,685.305,7,31,62
4,15 a 17 anos,435.976,"4,7","28,6","66,7"


,Area de residencia y grupos de edad,Poblacion total,Poblacion que asiste,Tasa de asistencia
0,Total pais,1.843.859,1.711.096,"92,8"
1,5 a 9 anos (T. P.),720.501,684.367,95
2,10 a 14 anos (T. P.),685.305,660.969,"96,4"
3,15 a 17 anos (T. P.),438.053,365.760,"83,5"
4,Area Urbana,1.078.702,1.020.066,"94,6"
5,5 a 9 anos (A. U.),412.098,400.694,"97,2"
6,10 a 14 anos (A. U.),402.657,392.542,"97,5"
7,15 a 17 anos (A. U.),263.947,226.830,"85,9"
8,Area Rural,765.157,691.030,"90,3"
9,5 a 9 anos (A. R.),308.403,283.673,92


,Grupos de edad,Total Pais,Consulto,No consulto
0,Total,399.814000,"61,8","38,2"
1,0 a 4 anos,140.096000,"74,5","25,5"
2,5 a 9 anos,128.891000,"57,9","42,1"
3,10 a 14 anos,73.879000,"51,3","48,7"
4,15 a 17 anos,56.948000,"53,3","46,7"


,Area de residencia y grupos de edad,Total pais,Hombres,Mujeres
0,Total Pais,2.511.100,1.338.145,1.172.955
1,T. P. 0 a 4 anos,"26,6",26,"27,2"
2,T. P. 5 a 9 anos,"28,7","29,3",28
3,T. P. 10 a 14 anos,"27,3","27,6","26,9"
4,T. P. 15 a 17 anos,"17,4","17,1","17,8"
5,Area Urbana,1.477.574,772.687,704.887
6,A. U. 0 a 4 anos,27,"26,6","27,5"
7,A. U. 5 a 9 anos,"27,9","28,1","27,7"
8,A. U. 10 a 14 anos,"27,3","27,5","26,9"
9,A. U. 15 a 17 anos,"17,9","17,8","17,9"


In [93]:
# assign path
path, dirs, files2 = next(os.walk("./data/indicadores_internet/"))
file_count = len(files2)

In [94]:
# create empty list
dataframes_list_indicadores_internet = []

In [95]:
# append datasets to the list
for i in range(file_count):
	temp_df = pd.read_csv("./data/indicadores_internet/"+files2[i], delimiter=';')
	dataframes_list_indicadores_internet.append(temp_df)

In [96]:
# display datasets
count2 = 0
for dataset in dataframes_list_indicadores_internet:
    display(dataset.style.set_table_attributes("style=display:inline").set_caption(files2[count2]))
    count2+=1

,Sexo y anos de estudio,Pob. de 10 anos y mas (2015),Pob. que utilizo internet (2015),Porc. en 2015,Pob. de 10 anos y mas (2016),Pob. que utilizo internet (2016),Porc. en 2016,Pob. de 10 anos y mas (2017),Pob. que utilizo internet (2017),Porc. en 2017,Pob. de 10 anos y mas (2018),Pob. que utilizo internet (2018),Porc. en 2018,Pob. de 10 anos y mas (2019),Pob. que utilizo internet (2019),Porc. en 2019,Pob. de 10 anos y mas (2020),Pob. que utilizo internet (2020),Porc. en 2020
0,Total Poblacion,5.305.163,2.637.531,"49,7",5.399.034,2.883.307,"53,4",5.493.661,3.355.295,"61,1",5.588.529,3.632.183,65,5.683.946,3.894.505,"68,5",5.779.774,4.274.577,74
1,T.P. Sin instruccion,116.686,-,-,127.108,-,-,127.376,8.413,"6,6",132.034,12.250,"9,3",134.871,14.203,"10,5",120.520,15.013,"12,5"
2,T.P. 1 a 6 anos,2.064.271,386.049,"18,7",2.148.418,509.564,"23,7",2.157.209,679.307,"31,5",2.130.943,768.882,"36,1",2.060.753,800.441,"38,8",2.062.967,1.013.183,"49,1"
3,T.P. 7 a 12 anos,2.182.417,1.423.758,"65,2",2.198.163,1.523.235,"69,3",2.211.624,1.719.180,"77,7",2.247.452,1.816.786,"80,8",2.331.284,1.962.112,"84,2",2.425.506,2.112.403,"87,1"
4,T.P. 13 a 18 anos,937.171,821.122,"87,6",921.098,844.292,"91,7",994.400,947.345,"95,3",1.075.714,1.033.202,96,1.153.635,1.116.014,"96,7",1.165.791,1.130.318,97
5,T.P. No disponible,4.618,2.348,"50,8",4.247,2.684,"63,2",3.052,1.050,"34,4",2.386,1.063,"44,6",3.403,1.735,51,4.990,3.660,"73,3"
6,Total Hombres,2.631.064,1.297.852,"49,3",2.701.271,1.432.465,53,2.750.083,1.672.417,"60,8",2.793.708,1.797.036,"64,3",2.808.890,1.910.221,68,2.869.238,2.092.847,"72,9"
7,T.H. Sin instruccion,43.997,-,-,47.446,-,-,50.030,-,-,55.655,-,-,54.898,-,-,50.873,7.351,"14,4"
8,T.H. 1 a 6 anos,1.020.983,195.917,"19,2",1.083.231,255.762,"23,6",1.075.692,334.296,"31,1",1.063.396,370.231,"34,8",1.013.962,386.998,"38,2",1.041.361,505.638,"48,6"
9,T.H. 7 a 12 anos,1.162.518,744.599,"64,1",1.156.751,798.900,"69,1",1.171.289,908.052,"77,5",1.183.229,952.404,"80,5",1.227.055,1.024.207,"83,5",1.275.574,1.095.001,"85,8"


,Area de residencia y grupos de edad,Estimacion (2015),Error muestral de la estimacion (2015),Porc. Coef. de variacion (2015),L.I. de Intervalo de confianza (95 Porc.) (2015),L.S. de Intervalo de confianza (95 Porc.) (2015),Estimacion (2016),Error muestral de la estimacion (2016),Porc. Coef. de variacion (2016),L.I. de Intervalo de confianza (95 Porc.) (2016),L.S. de Intervalo de confianza (95 Porc.) (2016),Estimacion (2017),Error muestral de la estimacion (2017),Porc. Coef. de variacion (2017),L.I. de Intervalo de confianza (95 Porc.) (2017),L.S. de Intervalo de confianza (95 Porc.) (2017),Estimacion (2018),Error muestral de la estimacion (2018),Porc. Coef. de variacion (2018),L.I. de Intervalo de confianza (95 Porc.) (2018),L.S. de Intervalo de confianza (95 Porc.) (2018),Estimacion (2019),Error muestral de la estimacion (2019),Porc. Coef. de variacion (2019),L.I. de Intervalo de confianza (95 Porc.) (2019),L.S. de Intervalo de confianza (95 Porc.) (2019),Estimacion (2020),Error muestral de la estimacion (2020),Porc. Coef. de variacion (2020),L.I. de Intervalo de confianza (95 Porc.) (2020),L.S. de Intervalo de confianza (95 Porc.) (2020)
0,Total Pais (T.P.),2.637.531,98.490000,"3,7",2.444.179,2.830.883,2.883.307,47.339000,"1,6",2.790.403,2.976.211,3.355.295,55.880000,"1,7",3.245.628,3.464.962,3.632.183,79.966000,"2,2",3.475.061,3.789.305,3.894.505,76.452000,2,3.744.288,4.044.722,4.274.577,88.929000,"2,1",4.099.844,4.449.310
1,10 - 14 anos (T.P.),206.761,13.836000,"6,7",179.599,233.923,203.672,9.083000,"4,5",185.845,221.499,233.732,10.875000,"4,7",212.389,255.075,245.151,12.415000,"5,1",220.758,269.544,281.194,15.084000,"5,4",251.556,310.832,420.430,19.728000,"4,7",381.667,459.193
2,15 - 19 anos (T.P.),508.959,22.135000,"4,3",465.505,552.413,511.682,15.606000,3,481.054,542.310,544.897,16.142000,3,513.218,576.576,562.539,21.835000,"3,9",519.636,605.442,564.147,21.863000,"3,9",521.189,607.105,622.711,24.103000,"3,9",575.351,670.071
3,20 - 24 anos (T.P.),487.475,28.535000,"5,9",431.456,543.494,503.931,15.452000,"3,1",473.606,534.256,548.499,16.994000,"3,1",515.147,581.851,572.560,23.623000,"4,1",526.144,618.976,578.455,22.197000,"3,8",534.842,622.068,565.720,22.210000,"3,9",522.081,609.359
4,25 - 29 anos (T.P.),386.731,18.909000,"4,9",349.610,423.852,427.888,13.836000,"3,2",400.734,455.042,481.957,15.551000,"3,2",451.438,512.476,505.026,21.325000,"4,2",463.126,546.926,543.419,20.384000,"3,8",503.368,583.470,535.850,22.909000,"4,3",490.837,580.863
5,30 - 34 anos (T.P.),335.147,18.665000,"5,6",298.504,371.790,369.479,11.892000,"3,2",346.141,392.817,419.198,13.669000,"3,3",392.372,446.024,415.750,17.605000,"4,2",381.159,450.341,458.215,17.719000,"3,9",423.400,493.030,463.516,19.788000,"4,3",424.636,502.396
6,35 anos y mas (T.P.),712.458,29.601000,"4,2",654.346,770.570,866.655,21.271000,"2,5",824.910,908.400,1.127.012,22.960000,2,1.081.952,1.172.072,1.331.157,33.574000,"2,5",1.265.190,1.397.124,1.469.075,32.196000,"2,2",1.405.814,1.532.336,1.666.350,36.209000,"2,2",1.595.205,1.737.495
7,Area Urbana (A.U.),2.015.284,86.555000,"4,3",1.845.362,2.185.206,2.191.307,41.585000,"1,9",2.109.694,2.272.920,2.458.878,45.647000,"1,9",2.369.293,2.548.463,2.625.531,65.941000,"2,5",2.495.965,2.755.097,2.799.691,62.495000,"2,2",2.676.897,2.922.485,2.982.703,71.845000,"2,4",2.841.537,3.123.869
8,10 - 14 anos (A.U.),157.050,11.904000,"7,6",133.681,180.419,158.494,8.453000,"5,3",141.905,175.083,179.440,9.921000,"5,5",159.970,198.910,175.990,10.449000,"5,9",155.460,196.520,208.626,13.469000,"6,5",182.161,235.091,279.335,15.473000,"5,5",248.933,309.737
9,15 - 19 anos (A.U.),365.257,19.794000,"5,4",326.399,404.115,359.894,14.109000,"3,9",332.204,387.584,361.231,13.935000,"3,9",333.883,388.579,372.443,19.018000,"5,1",335.075,409.811,373.156,18.290000,"4,9",337.220,409.092,403.468,20.333000,5,363.518,443.418


,Area de residencia y motivo de la utilizacion del servicio,Total (2015),Hombres (2015),Mujeres (2015),Total (2016),Hombres (2016),Mujeres (2016),Total (2017),Hombres (2017),Mujeres (2017),Total (2018),Hombres (2018),Mujeres (2018),Total (2019),Hombres (2019),Mujeres (2019),Total (2020),Hombres (2020),Mujeres (2020)
0,Total Pais (T.P.),2.637.531,1.297.852,1.339.679,2.883.307,1.432.465,1.450.842,3.355.295,1.672.417,1.682.878,3.632.183,1.797.036,1.835.147,3.894.505,1.910.221,1.984.284,4.274.577,2.092.847,2.181.730
1,Redes sociales (T.P.),"90,2","89,9","90,4","89,2","88,7","89,7","86,6","87,2","86,1","85,6","85,7","85,4","84,4","84,9","83,8","81,9","81,4","82,5"
2,Correos electronicos (T.P.),"46,4","46,6","46,2","48,2","48,2","48,2","44,5","45,8","43,2","40,1","40,3","39,8","40,2",41,"39,5","40,5","40,3","40,7"
3,Mensajeria instantanea (T.P.),"93,6","93,6","93,7","95,8","95,6",96,"96,5","96,5","96,4","97,2",97,"97,3","97,3","97,3","97,3","96,4","96,2","96,6"
4,Comunicaciones telefonicas (T.P.),"32,3","32,5","32,1","57,4",57,"57,8","78,6","78,7","78,4","77,2","77,1","77,4","81,7","81,7","81,8","80,4","79,3","81,3"
5,Productos y servicios (T.P.),"13,3","13,6",13,"31,2","30,9","31,5","30,2","29,9","30,6","38,4","38,1","38,7","44,9","44,2","45,5","37,1","36,1","38,1"
6,Salud y servicios de salud (T.P.),"8,5","6,9","10,1","21,1","17,1","24,9","24,1","19,7","28,5","29,4","24,8","33,9","35,2","29,7","40,6","37,3","32,3","42,1"
7,Servicios de instituciones del gobierno (T.P.),"7,9","7,7","8,1","11,5","11,3","11,7","11,6","11,4","11,8","14,1","14,1",14,"17,5","16,9",18,"23,8","22,4","25,1"
8,Comprar y/o vender productos o servicios (T.P.),"8,7","9,5","7,9","10,5","10,9",10,"9,5","10,4","8,7","11,4","12,1","10,8","16,9","17,5","16,2","15,9","15,9","15,8"
9,Transacciones bancarias (T.P.),"5,2","5,7","4,7","5,6","6,1","5,1","5,1","5,6","4,6","4,6",5,"4,2","6,3","6,8","5,8","7,1",8,"6,2"


,Area de residencia y lugar de acceso,Total (2015),Hombres (2015),Mujeres (2015),Total (2016),Hombres (2016),Mujeres (2016),Total (2017),Hombres (2017),Mujeres (2017),Total (2018),Hombres (2018),Mujeres (2018),Total (2019),Hombres (2019),Mujeres (2019),Total (2020),Hombres (2020),Mujeres (2020)
0,Total Pais,2.637.531,1.297.852,1.339.679,2.883.307,1.432.465,1.450.842,3.355.295,1.672.417,1.682.878,3.632.183,1.797.036,1.835.147,3.894.505,1.910.221,1.984.284,4.274.577,2.092.847,2.181.730
1,En el hogar (T.P.),"31,7","31,2","32,1","26,9","26,5","27,4",23,"23,3","22,7","26,8","26,5","27,1","30,7","30,1","31,3","41,8","40,8","42,7"
2,En el trabajo (T.P.),"18,6","20,3",17,20,"21,7","18,4","16,1","17,4","14,7","17,6",19,"16,2","20,1","21,9","18,4","16,8","18,3","15,4"
3,Institucion educativa (T.P.),"13,1",12,"14,3","11,7","10,4",13,"9,1","8,9","9,3","8,6",8,"9,3","10,4","9,2","11,6","2,8","2,6",3
4,Casa de otra persona (T.P.),"6,6","6,7","6,5","10,6","10,5","10,8","8,8","9,5","8,1","11,1","10,8","11,3","16,8","16,4","17,1","11,3","10,8","11,9"
5,Sitios comerciales (CYBER) (T.P.),"5,2","5,2","5,2","4,8","4,8","4,8","4,4","4,6","4,3","2,5","2,6","2,4","2,8","2,9","2,7","0,9","0,9","0,9"
6,Local de acceso comunitario (T.P.),"4,2","4,4",4,"7,9","7,9","7,9",7,"7,2","6,8","7,8","8,2","7,4","7,2","8,2","6,2","3,1","3,2","3,1"
7,A traves de un celular (T.P.),"94,2","94,5","93,9","95,5","95,4","95,6","97,8","97,9","97,7","98,6","98,5","98,6","98,7","98,8","98,5","97,3","96,9","97,6"
8,A traves de otros dispositivos (T.P.),7,"7,2","6,8",8,"8,2","7,8","6,2","6,4","6,1","3,9","4,2","3,6","7,8","8,1","7,6","8,2","7,9","8,5"
9,Area Urbana,2.015.284,966.456,1.048.828,2.191.307,1.063.767,1.127.540,2.458.878,1.207.484,1.251.394,2.625.531,1.284.813,1.340.718,2.799.691,1.355.599,1.444.092,2.982.703,1.435.077,1.547.626


,Area de residencia y acceso a la comunicacion,Ano 2015,Ano 2016,Ano 2017,Ano 2018,Ano 2019,Ano 2020
0,Total Pais (T.P.),1.665.279,1.729.864,1.756.991,1.795.109,1.863.684,1.867.423
1,Telefono linea fija (T.P.),"15,3","13,4","11,2","9,3","8,2","5,8"
2,Telefono movil (T.P.),"95,7","96,6","97,1","96,7","96,7","97,7"
3,Computadora/notebook (T.P.),"28,2","24,9","24,6","24,6","25,1","26,3"
4,Tableta o dispositivo similar (T.P.),"5,6","6,3","5,9","5,4","5,9",7
5,Acceso a internet (T.P.),"22,8","22,5","20,4","24,4","28,3","36,5"
6,Radio (T.P.),"80,3","76,1",76,"76,4","73,7",74
7,Televisor (T.P.),"92,7","90,9","90,4",91,90,"90,7"
8,Antena parabolica (T.P.),7,"4,6","3,1","3,1","2,2","1,7"
9,Tv cable (T.P.),"43,3","45,7","47,7","51,1","51,9","51,4"


...

,Sexo y grupos de edad,Estimacion (2015),Error muestral de la estimacion (2015),Porc. Coef. de variacion (2015),L.I. de Intervalo de confianza (95 Porc.) (2015),L.S. de Intervalo de confianza (95 Porc.) (2015),Estimacion (2016),Error muestral de la estimacion (2016),Porc. Coef. de variacion (2016),L.I. de Intervalo de confianza (95 Porc.) (2016),L.S. de Intervalo de confianza (95 Porc.) (2016),Estimacion (2017),Error muestral de la estimacion (2017),Porc. Coef. de variacion (2017),L.I. de Intervalo de confianza (95 Porc.) (2017),L.S. de Intervalo de confianza (95 Porc.) (2017),Estimacion (2018),Error muestral de la estimacion (2018),Porc. Coef. de variacion (2018),L.I. de Intervalo de confianza (95 Porc.) (2018),L.S. de Intervalo de confianza (95 Porc.) (2018),Estimacion (2019),Error muestral de la estimacion (2019),Porc. Coef. de variacion (2019),L.I. de Intervalo de confianza (95 Porc.) (2019),L.S. de Intervalo de confianza (95 Porc.) (2019),Estimacion (2020),Error muestral de la estimacion (2020),Porc. Coef. de variacion (2020),L.I. de Intervalo de confianza (95 Porc.) (2020),L.S. de Intervalo de confianza (95 Porc.) (2020)
0,Total Poblacion (T.P),2.637.531,98.490000,"3,7",2.444.179,2.830.883,2.883.307,47.339000,"1,6",2.790.403,2.976.211,3.355.295,55.880000,"1,7",3.245.628,3.464.962,3.632.183,79.966000,"2,2",3.475.061,3.789.305,3.894.505,76.452000,2,3.744.288,4.044.722,4.274.577,88.929000,"2,1",4.099.844,4.449.310
1,10 - 14 anos (T.P),206.761,13.836000,"6,7",179.599,233.923,203.672,9.083000,"4,5",185.845,221.499,233.732,10.875000,"4,7",212.389,255.075,245.151,12.415000,"5,1",220.758,269.544,281.194,15.084000,"5,4",251.556,310.832,420.430,19.728000,"4,7",381.667,459.193
2,15 - 19 anos (T.P),508.959,22.135000,"4,3",465.505,552.413,511.682,15.606000,3,481.054,542.310,544.897,16.142000,3,513.218,576.576,562.539,21.835000,"3,9",519.636,605.442,564.147,21.863000,"3,9",521.189,607.105,622.711,24.103000,"3,9",575.351,670.071
3,20 - 24 anos (T.P),487.475,28.535000,"5,9",431.456,543.494,503.931,15.452000,"3,1",473.606,534.256,548.499,16.994000,"3,1",515.147,581.851,572.560,23.623000,"4,1",526.144,618.976,578.455,22.197000,"3,8",534.842,622.068,565.720,22.210000,"3,9",522.081,609.359
4,25-29 (T.P),386.731,18.909000,"4,9",349.610,423.852,427.888,13.836000,"3,2",400.734,455.042,481.957,15.551000,"3,2",451.438,512.476,505.026,21.325000,"4,2",463.126,546.926,543.419,20.384000,"3,8",503.368,583.470,535.850,22.909000,"4,3",490.837,580.863
5,30 - 34 anos (T.P),335.147,18.665000,"5,6",298.504,371.790,369.479,11.892000,"3,2",346.141,392.817,419.198,13.669000,"3,3",392.372,446.024,415.750,17.605000,"4,2",381.159,450.341,458.215,17.719000,"3,9",423.400,493.030,463.516,19.788000,"4,3",424.636,502.396
6,35 anos y mas (T.P),712.458,29.601000,"4,2",654.346,770.570,866.655,21.271000,"2,5",824.910,908.400,1.127.012,22.960000,2,1.081.952,1.172.072,1.331.157,33.574000,"2,5",1.265.190,1.397.124,1.469.075,32.196000,"2,2",1.405.814,1.532.336,1.666.350,36.209000,"2,2",1.595.205,1.737.495
7,Total Hombres (T.H),1.297.852,48.656000,"3,7",1.202.333,1.393.371,1.432.465,26.585000,"1,9",1.380.290,1.484.640,1.672.417,31.056000,"1,9",1.611.469,1.733.365,1.797.036,42.750000,"2,4",1.713.038,1.881.034,1.910.221,42.656000,"2,2",1.826.409,1.994.033,2.092.847,46.432000,"2,2",2.001.614,2.184.080
8,10 - 14 anos (T.H),106.114,8.680000,"8,2",89.073,123.155,109.646,6.011000,"5,5",97.849,121.443,124.255,7.521000,"6,1",109.494,139.016,124.956,8.556000,"6,8",108.144,141.768,144.947,10.364000,"7,2",124.583,165.311,225.393,12.631000,"5,6",200.576,250.210
9,15 - 19 anos (T.H),248.234,10.753000,"4,3",227.125,269.343,257.539,10.253000,4,237.417,277.661,287.141,10.783000,"3,8",265.979,308.303,302.761,14.399000,"4,8",274.468,331.054,289.451,14.802000,"5,1",260.368,318.534,323.022,15.937000,"4,9",291.708,354.336


,Departamento,Pob. de 10 anos y mas (2015),Pob. que utilizo internet (2015),Porc. en 2015,Pob. de 10 anos y mas (2016),Pob. que utilizo internet (2016),Porc. en 2016,Pob. de 10 anos y mas (2017),Pob. que utilizo internet (2017),Porc. en 2017,Pob. de 10 anos y mas (2018),Pob. que utilizo internet (2018),Porc. en 2018,Pob. de 10 anos y mas (2019),Pob. que utilizo internet (2019),Porc. en 2019,Pob. de 10 anos y mas (2020),Pob. que utilizo internet (2020),Porc. en 2020
0,Total pais,5.305.163,2.637.531,"49,7",5.399.034,2.883.307,"53,4",5.493.661,3.355.295,"61,1",5.588.529,3.632.183,65,5.683.946,3.894.505,"68,5",5.779.774,4.274.577,74
1,Asuncion,430.364,309.130,"71,8",432.542,312.689,"72,3",440.452,340.975,"77,4",429.335,347.204,"80,9",430.505,368.220,"85,5",430.105,373.607,"86,9"
2,San Pedro,314.591,70.080,"22,3",319.531,99.240,"31,1",330.995,124.537,"37,6",336.833,144.638,"42,9",343.274,161.698,"47,1",344.658,205.720,"59,7"
3,Caaguazu,424.550,147.972,"34,9",417.977,153.898,"36,8",412.752,200.421,"48,6",424.338,226.307,"53,3",443.891,264.263,"59,5",445.673,308.793,"69,3"
4,Caazapa,144.732,45.789,"31,6",141.048,49.310,35,146.742,55.749,38,150.871,69.202,"45,9",150.016,84.688,"56,5",157.332,99.567,"63,3"
5,Itapua,442.169,243.974,"55,2",465.838,244.253,"52,4",481.171,278.586,"57,9",482.583,311.974,"64,6",483.802,340.030,"70,3",502.196,368.234,"73,3"
6,Alto Parana,602.684,320.847,"53,2",610.343,340.622,"55,8",617.996,422.222,"68,3",643.635,450.690,70,643.489,464.824,"72,2",667.987,519.117,"77,7"
7,Central,1.602.211,963.452,"60,1",1.647.154,1.077.568,"65,4",1.683.387,1.210.058,"71,9",1.718.343,1.303.631,"75,9",1.770.973,1.356.899,"76,6",1.792.497,1.431.630,"79,9"
8,Resto,1.343.862,536.287,"39,9",1.364.601,605.727,"44,4",1.380.166,722.747,"52,4",1.402.591,778.537,"55,5",1.417.996,853.883,"60,2",1.439.326,967.909,"67,2"


,Sexo y grupos de edad,Pob. de 10 anos y mas (2015),Pob. que utilizo internet (2015),Porc. en 2015,Pob. de 10 anos y mas (2016),Pob. que utilizo internet (2016),Porc. en 2016,Pob. de 10 anos y mas (2017),Pob. que utilizo internet (2017),Porc. en 2017,Pob. de 10 anos y mas (2018),Pob. que utilizo internet (2018),Porc. en 2018,Pob. de 10 anos y mas (2019),Pob. que utilizo internet (2019),Porc. en 2019,Pob. de 10 anos y mas (2020),Pob. que utilizo internet (2020),Porc. en 2020
0,Total Poblacion,5.305.163,2.637.531,"49,7",5.399.034,2.883.307,"53,4",5.493.661,3.355.295,"61,1",5.588.529,3.632.183,65,5.683.946,3.894.505,"68,5",5.779.774,4.274.577,74
1,T.P. 10-14 anos,647.915,206.761,"31,9",688.268,203.672,"29,6",672.091,233.732,"34,8",691.482,245.151,"35,5",694.975,281.194,"40,5",685.305,420.430,"61,3"
2,T.P. 15-19 anos,727.386,508.959,70,702.997,511.682,"72,8",699.085,544.897,"77,9",692.079,562.539,"81,3",678.847,564.147,"83,1",711.131,622.711,"87,6"
3,T.P. 20-24 anos,643.209,487.475,"75,8",624.325,503.931,"80,7",641.736,548.499,"85,5",646.139,572.560,"88,6",637.991,578.455,"90,7",632.933,565.720,"89,4"
4,T.P. 25-29 anos,536.145,386.731,"72,1",553.569,427.888,"77,3",569.126,481.957,"84,7",563.658,505.026,"89,6",592.237,543.419,"91,8",585.159,535.850,"91,6"
5,T.P. 30-34 anos,493.593,335.147,"67,9",515.299,369.479,"71,7",518.502,419.198,"80,8",489.633,415.750,"84,9",522.825,458.215,"87,6",512.439,463.516,"90,5"
6,T.P. 35 anos y mas,2.256.915,712.458,"31,6",2.314.576,866.655,"37,4",2.393.121,1.127.012,"47,1",2.505.538,1.331.157,"53,1",2.557.071,1.469.075,"57,5",2.652.807,1.666.350,"62,8"
7,Total Hombres,2.631.064,1.297.852,"49,3",2.701.271,1.432.465,53,2.750.083,1.672.417,"60,8",2.793.708,1.797.036,"64,3",2.808.890,1.910.221,68,2.869.238,2.092.847,"72,9"
8,T.H. 10-14 anos,335.341,106.114,"31,6",360.808,109.646,"30,4",348.515,124.255,"35,7",349.149,124.956,"35,8",350.629,144.947,"41,3",369.419,225.393,61
9,T.H. 15-19 anos,372.431,248.234,"66,7",366.273,257.539,"70,3",370.677,287.141,"77,5",372.268,302.761,"81,3",352.407,289.451,"82,1",374.640,323.022,"86,2"


,Area de residencia y ocupacion principal,Pob. de 10 anos y mas (2015),Pob. que utilizo internet (2015),Porc. en 2015,Pob. de 10 anos y mas (2016),Pob. que utilizo internet (2016),Porc. en 2016,Pob. de 10 anos y mas (2017),Pob. que utilizo internet (2017),Porc. en 2017,Pob. de 10 anos y mas (2018),Pob. que utilizo internet (2018),Porc. en 2018,Pob. de 10 anos y mas (2019),Pob. que utilizo internet (2019),Porc. en 2019,Pob. de 10 anos y mas (2020),Pob. que utilizo internet (2020),Porc. en 2020
0,Total Pais,3.061.380,1.663.998,"54,4",3.122.747,1.886.274,"60,4",3.228.636,2.208.701,"68,4",3.317.775,2.418.947,"72,9",3.422.331,2.613.368,"76,4",3.421.436,2.719.800,"79,5"
1,"Miembros Poder Ejecutivo, Legislativo, Judicial y Personal Directivo (T.P.)",101.508,84.812,"83,6",122.179,104.607,"85,6",122.825,114.779,"93,4",139.231,133.432,"95,8",149.217,142.668,"95,6",133.445,129.929,"97,4"
2,Profesionales Cientificos e Intelectuales (T.P.),239.881,212.497,"88,6",222.827,203.213,"91,2",204.303,198.625,"97,2",215.687,211.233,"97,9",250.850,247.533,"98,7",265.518,261.571,"98,5"
3,Tecnicos y Profesionales de Nivel Medio (T.P.),236.935,201.369,85,228.103,200.897,"88,1",228.817,212.602,"92,9",253.891,245.366,"96,6",240.874,234.324,"97,3",222.113,215.736,"97,1"
4,Empleados de Oficina (T.P.),158.328,135.419,"85,5",146.901,134.199,"91,4",151.098,142.198,"94,1",153.639,150.810,"98,2",173.649,170.898,"98,4",150.553,143.544,"95,3"
5,Trabajadores de Servicios y Vendedores de Comercios y Mercados (T.P.),618.227,359.536,"58,2",607.323,414.929,"68,3",634.556,472.063,"74,4",653.313,526.874,"80,6",655.014,537.416,82,687.888,585.106,"85,1"
6,Agricultores y Trabajadores Agropecuarios y Pesqueros (T.P.),509.711,87.548,"17,2",554.527,120.544,"21,7",551.060,164.208,"29,8",551.185,186.475,"33,8",555.241,210.384,"37,9",591.420,293.168,"49,6"
7,"Oficiales, Operarios y Artesanos (T.P.)",453.948,233.960,"51,5",456.944,270.873,"59,3",482.286,335.383,"69,5",493.675,370.174,75,490.692,388.734,"79,2",514.153,416.583,81
8,Operadores de instalaciones y maquinas y montadores (T.P.),152.360,85.689,"56,2",153.534,103.937,"67,7",159.104,124.054,78,157.148,130.798,"83,2",173.397,154.439,"89,1",159.328,142.616,"89,5"
9,Trabajadores no calificados (T.P.),581.734,255.655,"43,9",623.359,326.394,"52,4",681.068,432.527,"63,5",691.545,455.324,"65,8",717.644,511.219,"71,2",687.266,524.055,"76,3"


,Area de residencia y grupos de edad,Pob. de 10 anos y mas (2015),Pob. que utilizo internet (2015),Porc. en 2015,Pob. de 10 anos y mas (2016),Pob. que utilizo internet (2016),Porc. en 2016,Pob. de 10 anos y mas (2017),Pob. que utilizo internet (2017),Porc. en 2017,Pob. de 10 anos y mas (2018),Pob. que utilizo internet (2018),Porc. en 2018,Pob. de 10 anos y mas (2019),Pob. que utilizo internet (2019),Porc. en 2019,Pob. de 10 anos y mas (2020),Pob. que utilizo internet (2020),Porc. en 2020
0,Total Pais,5.305.163,2.637.531,"49,7",5.399.034,2.883.307,"53,4",5.493.661,3.355.295,"61,1",5.588.529,3.632.183,65,5.683.946,3.894.505,"68,5",5.779.774,4.274.577,74
1,T.P. 10-14 anos,647.915,206.761,"31,9",688.268,203.672,"29,6",672.091,233.732,"34,8",691.482,245.151,"35,5",694.975,281.194,"40,5",685.305,420.430,"61,3"
2,T.P. 15-19 anos,727.386,508.959,70,702.997,511.682,"72,8",699.085,544.897,"77,9",692.079,562.539,"81,3",678.847,564.147,"83,1",711.131,622.711,"87,6"
3,T.P. 20-24 anos,643.209,487.475,"75,8",624.325,503.931,"80,7",641.736,548.499,"85,5",646.139,572.560,"88,6",637.991,578.455,"90,7",632.933,565.720,"89,4"
4,T.P. 25-29 anos,536.145,386.731,"72,1",553.569,427.888,"77,3",569.126,481.957,"84,7",563.658,505.026,"89,6",592.237,543.419,"91,8",585.159,535.850,"91,6"
5,T.P. 30-34 anos,493.593,335.147,"67,9",515.299,369.479,"71,7",518.502,419.198,"80,8",489.633,415.750,"84,9",522.825,458.215,"87,6",512.439,463.516,"90,5"
6,T.P. 35 anos y mas,2.256.915,712.458,"31,6",2.314.576,866.655,"37,4",2.393.121,1.127.012,"47,1",2.505.538,1.331.157,"53,1",2.557.071,1.469.075,"57,5",2.652.807,1.666.350,"62,8"
7,Area Urbana,3.281.047,2.015.284,"61,4",3.362.096,2.191.307,"65,2",3.443.556,2.458.878,"71,4",3.525.168,2.625.531,"74,5",3.607.037,2.799.691,"77,6",3.689.195,2.982.703,"80,8"
8,T.U. 10-14 anos,351.433,157.050,"44,7",377.251,158.494,42,379.241,179.440,"47,3",391.266,175.990,45,404.118,208.626,"51,6",402.657,279.335,"69,4"
9,T.U. 15-19 anos,447.864,365.257,"81,6",435.759,359.894,"82,6",420.492,361.231,"85,9",424.935,372.443,"87,6",409.397,373.156,"91,1",442.470,403.468,"91,2"


In [98]:
# we will start to make the map here

In [99]:
# import necesary packages
import folium
from folium import plugins
import webbrowser